# Fashion MNIST

### [2024.01.28] 이상혁

### 개요
- [출처] 딥러닝 파이토치 교과서 5장 2절의 합성곱 신경망 맛보기
- fashion_mnist 데이터셋을 사용하여 합성곱 신경망을 직접 구현해보자.
- fashion_mnist 데이터셋은 토치비전에 내장된 예제 데이터로 운동화, 셔츠, 샌들 같은 작은 이미지의 모음이며, 기본 MNIST 데이터셋처럼 열 가지로 분류될 수 있는 28X28 픽셀의 이미지 7만 개로 되어 있다.
- 훈련 데이터는 0부터 255 사이의 값을 갖는 28X28 크기의 넘파이 배열이다.

### 0. GPU Configuration

💻빠른 연산을 위해 Tensorflow에서 **GPU**를 사용할 수 있도록 설정해준다.

In [1]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


GPU 설정이 마무리 되었다. 이제 시드를 설정하고서 본격적으로 시작해보자.

In [2]:
# 시드 고정
torch.manual_seed(128)

if device == 'cuda':
    torch.cuda.manual_seed_all(128)

### 1. Libraries

In [15]:
import warnings
warnings.filterwarnings('ignore')

import os
import sys
import time
import random
import datetime

import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms # data pre-processing
from torch.utils.data import Dataset, DataLoader

### 2. Load Data

이제 데이터를 불러오자. 토치비전으로 다운 받을 수 있다.

In [4]:
train_data = torchvision.datasets.FashionMNIST('C:/Python_Programs/Pytorch/1. Fashion MNIST', download=True,
                                               transform=transforms.Compose([transforms.ToTensor()]))
test_data =  torchvision.datasets.FashionMNIST('C:/Python_Programs/Pytorch/1. Fashion MNIST', download=True, train=False,
                                               transform=transforms.Compose([transforms.ToTensor()]))

다운 받은 데이터를 매모리로 불러오기 위해 데이터로더(DataLoader)에 전달한다.

In [5]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=100)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=100)

batch_size=100으로 지정해서 100개 단위로 데이터를 묶어서 불러왔다.

이번에는 우리가 다루게 될 이미지를 자세히 살펴보자. 먼저 20개 정도의 이미지를 레이블 정보와 함께 출력해보자.

In [ ]:
# label
labels_map = {0 : 'T-Shirt', 1 : 'Trouser', 2 : 'Pullover', 3 : 'Dress', 4 : 'Coat', 
              5 : 'Sandal', 6 : 'Shirt', 7 : 'Sneaker', 8 : 'Bag', 9 : 'Ankle Boot'}

# plot
fig = plt.figure(figsize=(8, 8))
columns = 4
rows = 5

for i in range(1, columns*rows+1):
    img_xy = np.random.randint(len(train_loader)) # randomly choose 1 integer
    img = train_data[img_xy][0][0,:,:] # randomly choose 1 image
    fig.add_subplot(rows, columns, i)
    plt.title(labels_map[train_data[img_xy][1]]) # title with labels_map
    plt.axis('off')
    plt.imshow(img, cmap='gray')
plt.show()

### 3. Model

#### 3-1. Simple Neural Network

먼저 일반적인 심층 심경망을 생성해서 학습시켜보자. 먼저 클래스로 신경망을 구현해보자.

In [21]:
class FashionDNN(nn.Module):
    def __init__(self):
        super(FashionDNN, self).__init__()
        self.fc1 = nn.Linear(in_features=784, out_features=256)
        self.drop = nn.Dropout(0.25)
        self.fc2 = nn.Linear(in_features=256, out_features=128)
        self.fc3 = nn.Linear(in_features=128, out_features=10)

    def forward(self, input_data):
        out = input_data.view(-1, 784)
        out = F.relu(self.fc1(out))
        out = self.drop(out)
        out = F.relu(self.fc2(out))
        out = self.fc3(out)

        return out

신경망의 구조는 다음과 같다.
1. 선형회귀 : 784 크기의 입력을 받으면, 256 크기로 출력한다.
2. 드롭아웃 : 과적합 방지를 위한 계층으로, 25%의 뉴런을 0으로 만들고 75%에 해당하는 뉴런은 (1/(1-0.7))만큼 키운다.
3. 선형회귀 : 256 크기의 입력을 받으면, 128 크기로 출력한다.
4. 선형회귀 : 128 크기의 입력을 받으면, 10 크기로 출력한다. 이는 10개의 레이블로 구분하기 위함이다.


순전파의 경우는 다음 과정을 따른다.

1. view : 두번째 차원을 784로 고정 
2. 첫번째 선형회귀 층 통과 -> 렐루 함수 통과
3. 드롭아웃 계층 통과
4. 두번째 선형회귀 층 통과 -> 렐루 함수 통과 
5. 세번째 선형회귀층 통과

⚠ 이때 렐루 함수에는 두 가지 사용법이 있다.
- torch.nn.functional.relu() : 순전파 메서드인 forward()에서 사용
- torch.nn.ReLU() : 초기화 메서드인 init()에서 사용, nn.Sequential() 안에서 사용하려면 nn.ReLU()를 사용하면 됨

이제 학습을 위한 손실함수, 학습률, 옵티마이저를 정의하자.

In [22]:
learning_rate = 0.001

In [23]:
# model
model = FashionDNN()
model.to(device)

FashionDNN(
  (fc1): Linear(in_features=784, out_features=256, bias=True)
  (drop): Dropout(p=0.25, inplace=False)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)
)

In [24]:
# loss and optimizer
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

이제 학습을 진행하자.

In [25]:
num_epochs = 5
count = 0
loss_list = []
iteration_list = []
accuracy_list = []

predictions_list = []
labels_list = []

for epoch in range(num_epochs):
    for images, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
        images, labels = images.to(device), labels.to(device)
    
        train = Variable(images.view(100, 1, 28, 28))
        labels = Variable(labels)
        
        outputs = model(train)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        count += 1
    
        if (count % 50) == 0:    
            total = 0
            correct = 0        
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                labels_list.append(labels)            
                test = Variable(images.view(100, 1, 28, 28))            
                outputs = model(test)            
                predictions = torch.max(outputs, 1)[1].to(device)
                predictions_list.append(predictions)
                correct += (predictions == labels).sum()            
                total += len(labels)
            
            accuracy = correct * 100 / total
            loss_list.append(loss.data)
            iteration_list.append(count)
            accuracy_list.append(accuracy)
        
    # 매 epoch의 끝에서 손실과 정확도 출력
    tqdm.write("Epoch: {}, Loss: {:.2f}, Accuracy: {:.2f}%".format(epoch+1, loss.data, accuracy))


Epoch 1/5:   0%|          | 0/600 [00:00<?, ?it/s]

Epoch 1/5: 100%|██████████| 600/600 [00:27<00:00, 21.69it/s]


Epoch: 1, Loss: 0.41, Accuracy: 83.39%


Epoch 2/5: 100%|██████████| 600/600 [00:27<00:00, 21.93it/s]


Epoch: 2, Loss: 0.35, Accuracy: 85.46%


...

Epoch: 3, Loss: 0.23, Accuracy: 85.81%


Epoch 4/5: 100%|██████████| 600/600 [00:27<00:00, 21.89it/s]


Epoch: 4, Loss: 0.31, Accuracy: 86.24%


Epoch 5/5: 100%|██████████| 600/600 [00:27<00:00, 21.91it/s]

Epoch: 5, Loss: 0.27, Accuracy: 86.67%


- tqdm을 사용해서 학습 추이를 확인할 수 있도록 했다.
- 정확도는 약 86% 정도로 나쁘지 않다고 본다.

#### 3-2. CNN

이번에는 이미지 데이터에 특화된 CNN을 모델로 만들어보자.

In [32]:
class FashionCNN(nn.Module):    
    def __init__(self):
        super(FashionCNN, self).__init__()        
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )       
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )        
        self.fc1 = nn.Linear(in_features=64*6*6, out_features=600)
        self.drop = nn.Dropout2d(0.25)
        self.fc2 = nn.Linear(in_features=600, out_features=120)
        self.fc3 = nn.Linear(in_features=120, out_features=10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.drop(out)
        out = self.fc2(out)
        out = self.fc3(out)       
        return out

합성곱 신경망과 전결합층에 대해서 자세히 알아보자.
1. 합성곱층 : 필터 높이 $\times$ 너비 크기의 이미지를 처음부터 끝까지 훑으면서 값을 출력한다. 여기서 파라미터를 조금만 더 살펴보자.
- in_channels : 입력 채널의 수를 의미한다. 흑백일 때는 1, RGB일 때는 3으로 설정한다. 우리가 다루는 이미지 데이터는 흑백이기 때문에 1로 설정
- out_channels : 출력 채널의 수를 의미한다.
- kernel_size : 커널의 크기이며, 커널의 파라미터가 CNN의 학습 대상이다.
- padding : 패딩 크기를 의미하며, 합성곱 연선을 수행하기 이전에 입력 데이터의 크기를 키운다.

2. 배치 정규화 계층 : 데이터의 분포를 평균과 분산을 이용하여 정규화하는 것이다.

3. 맥스 풀링계층 : 이미지 크기를 축소시키는 용도로 사용한다. 파라미터를 살펴보자
- kernel_size : 가중치 행렬을 의미한다.
- stride : 이동 간격

4. 선형 회귀 계층 : 이미지 형태의 데이터를 배열 형태로 변환해야 한다. 
- in_features : 패딩, 스트라이드 등을 적용했기 때문에 이에 따른 출력 차원을 계산해야 한다.
- 순전파에서 이를 해결하기 위해 out.view(out.size(0), -1)을 사용한다. out.size(0)은 결국 100을 의미한다.

5. 드롭아웃

6. 선형 회귀

7. 선형 회귀

이제 학습을 진행해보자. 코드는 이전과 똑같다.

In [33]:
# learning rate
learning_rate = 0.001

# model
model = FashionCNN()
model.to(device)
print(model)

# loss and optimizer
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

FashionCNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=2304, out_features=600, bias=True)
  (drop): Dropout2d(p=0.25, inplace=False)
  (fc2): Linear(in_features=600, out_features=120, bias=True)
  (fc3): Linear(in_features=120, out_features=10, bias=True)
)


In [34]:
num_epochs = 5
count = 0
loss_list = []
iteration_list = []
accuracy_list = []

predictions_list = []
labels_list = []

for epoch in range(num_epochs):
    for images, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
        images, labels = images.to(device), labels.to(device)
    
        train = Variable(images.view(100, 1, 28, 28))
        labels = Variable(labels)
        
        outputs = model(train)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        count += 1
    
        if (count % 50) == 0:    
            total = 0
            correct = 0        
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                labels_list.append(labels)            
                test = Variable(images.view(100, 1, 28, 28))            
                outputs = model(test)            
                predictions = torch.max(outputs, 1)[1].to(device)
                predictions_list.append(predictions)
                correct += (predictions == labels).sum()            
                total += len(labels)
            
            accuracy = correct * 100 / total
            loss_list.append(loss.data)
            iteration_list.append(count)
            accuracy_list.append(accuracy)
        
    # 매 epoch의 끝에서 손실과 정확도 출력
    tqdm.write("Epoch: {}, Loss: {:.2f}, Accuracy: {:.2f}%".format(epoch+1, loss.data, accuracy))


Epoch 1/5: 100%|██████████| 600/600 [00:38<00:00, 15.54it/s]


Epoch: 1, Loss: 0.28, Accuracy: 87.41%


Epoch 2/5: 100%|██████████| 600/600 [00:38<00:00, 15.64it/s]


Epoch: 2, Loss: 0.25, Accuracy: 86.66%


Epoch 3/5: 100%|██████████| 600/600 [00:39<00:00, 15.26it/s]


Epoch: 3, Loss: 0.17, Accuracy: 88.94%


Epoch 4/5: 100%|██████████| 600/600 [00:36<00:00, 16.55it/s]


Epoch: 4, Loss: 0.15, Accuracy: 90.35%


Epoch 5/5: 100%|██████████| 600/600 [00:36<00:00, 16.53it/s]

Epoch: 5, Loss: 0.16, Accuracy: 90.22%


- 정확도가 무려 90%나 나오고 있다.
- 기본 심층 신경망보다는 약 4% 정도 더 좋은 성능을 보이고 있다.